In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

['Year_wise_batting_stats_all_ODI_players(30-05-19).csv']


In [2]:
data = pd.read_csv("../input/Year_wise_batting_stats_all_ODI_players(30-05-19).csv")
data = data.fillna(0)
data.head()

,country,player_name,0s,100s,50s,Avg,Ca,HS,Inns,Mat,NO,Runs,S/R,St,Year
0,AFG,Gulbadin Naib,1,0,0,0.00,0,0,1,1,0,0,0.00,0,2011
1,AFG,Gulbadin Naib,0,0,0,12.25,1,22,4,5,0,49,72.06,0,2012
2,AFG,Gulbadin Naib,0,0,0,0.00,0,21*,1,2,1,21,140.00,0,2013
3,AFG,Gulbadin Naib,0,0,0,27.33,0,31,4,4,1,82,103.80,0,2014
4,AFG,Gulbadin Naib,2,0,0,0.00,3,0,2,3,0,0,0.00,0,2015


In [3]:
data.dtypes

country         object
player_name     object
0s               int64
100s             int64
50s              int64
Avg            float64
Ca               int64
HS              object
Inns             int64
Mat              int64
NO               int64
Runs             int64
S/R            float64
St               int64
Year            object
dtype: object

<p>We can see that the highscore each year is an object. First thing's First. Let's convert that into an int64 dtype. Some of the highscores have an * (asterisk) next to them signifying that they were still not out when the innings got over. Let's remove all the asterisks as they are unimportant for our future needs. </p>

In [4]:
data['HS'] = data['HS'].map(lambda x: int(str(x).rstrip('*')))

In [5]:
data.dtypes

country         object
player_name     object
0s               int64
100s             int64
50s              int64
Avg            float64
Ca               int64
HS               int64
Inns             int64
Mat              int64
NO               int64
Runs             int64
S/R            float64
St               int64
Year            object
dtype: object

<h3> Overall Performance Dataframe </h3>
<p> Let's make a separate dataframe for the overall performance of players </p>

In [6]:
op = data.loc[data['Year'] == 'Overall']
op = op.drop(columns = "Year")

Outs is the number of times that player has gotten out in his career.

In [7]:
op["Outs"] = op["Inns"]-op["NO"]

"O/I" is the ratio of the number of times the batsman has got out by the number of innings played

In [8]:
op["O/I"] = op["Outs"]/op["Inns"]

In [9]:
op.head()

,country,player_name,0s,100s,50s,Avg,Ca,HS,Inns,Mat,NO,Runs,S/R,St,Outs,O/I
9,AFG,Gulbadin Naib,5,0,5,21.84,11,82,46,55,8,830,71.24,0,38,0.826087
13,AFG,Afsar Zazai,3,0,2,17.60,20,60,16,17,1,264,54.10,2,15,0.937500
21,AFG,Najibullah Zadran,2,1,9,30.24,34,104,51,56,9,1270,90.39,0,42,0.823529
31,AFG,Noor Ali Zadran,3,1,7,25.07,15,114,47,48,1,1153,63.35,0,46,0.978723
33,AFG,Karim Janat,0,0,0,9.00,0,9,1,1,0,9,27.27,0,1,1.000000


<h3> The hard-hitters </h3>
Let's Find out the players who have played more than 30 innings in ODI matches who have a good strike rate.

In [62]:
hard_hitters = op.loc[(op["Inns"]>30) & (op["S/R"]>80)].sort_values(by='S/R', ascending=False)
hard_hitters.head(10)

,country,player_name,0s,100s,50s,Avg,Ca,HS,Inns,Mat,NO,Runs,S/R,St,Outs,O/I
11445,WIN,Andre Russell,5,0,4,28.51,11,92,44,52,9,998,130.46,0,35,0.795455
11102,UAE,Mohammad Naveed,2,0,0,16.54,6,45,34,39,10,397,122.91,0,24,0.705882
1109,AUS,Glenn Maxwell,8,1,19,33.33,61,102,90,100,9,2700,121.95,0,81,0.900000
2990,ENG,Jos Butler,12,8,18,41.54,159,150,108,131,23,3531,119.57,29,85,0.787037
7480,PAK,Shahid Afridi,30,6,39,23.58,127,124,369,398,27,8064,117.01,0,342,0.926829
6474,NZL,Luke Ronchi,7,1,4,23.68,105,170,68,85,9,1397,114.51,12,59,0.867647
4460,IND,Yusuf Pathan,7,2,3,27.00,17,123,41,57,11,810,113.60,0,30,0.731707
10354,SRL,Thisara Perera,11,1,10,20.45,58,140,121,154,16,2147,112.29,0,105,0.867769
2362,CAN,Rizwan Cheema,3,0,6,24.65,15,94,32,33,1,764,111.53,0,31,0.968750
7100,NZL,Corey Anderson,4,1,4,27.73,11,131,44,49,4,1109,108.73,0,40,0.909091


In [78]:
trace1 =go.Scatter(
                    x = hard_hitters["Inns"],
                    y = hard_hitters["S/R"],
                    mode = "markers",
                    name = "Name",
                    marker = dict(color=(
            (hard_hitters["S/R"] > 110)            
        ).astype('int'),
        colorscale=[[0, 'yellow'], [1, 'red']]),
                    text= hard_hitters["player_name"])

data = [trace1]
layout = dict(title = 'The Hard Hitters of ODI Cricket',
              xaxis= dict(title= 'Number of Matches',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Strike Rate',ticklen= 5,zeroline= False),
              hovermode= 'closest',
             )
fig = dict(data = data, layout = layout)
iplot(fig)

In [109]:
consistent = op.loc[op["Inns"]>30].sort_values(by='Avg', ascending=False)
consistent.head(10)

,country,player_name,0s,100s,50s,Avg,Ca,HS,Inns,Mat,NO,Runs,S/R,St,Outs,O/I
5977,NED,Ryan ten Doeschate,1,5,9,67.00,13,119,32,33,9,1541,87.71,0,23,0.718750
4951,IND,Virat Kohli,12,41,49,59.58,111,183,219,227,37,10843,92.96,0,182,0.831050
828,AUS,Michael Bevan,5,6,46,53.58,69,108,196,232,67,6912,74.16,0,129,0.658163
9702,SAF,A B de Villiers,7,25,53,53.50,176,176,218,228,39,9577,101.10,5,179,0.821101
7601,PAK,Babar Azam,2,9,12,51.68,31,125,62,64,9,2739,85.97,0,53,0.854839
8538,PAK,Fakhar Zaman,3,4,10,51.31,16,210,36,36,4,1642,98.15,0,32,0.888889
3116,ENG,Jonathan Trott,5,4,22,51.25,14,137,65,68,10,2819,77.06,0,55,0.846154
11959,WIN,Shai Hope,4,6,10,51.07,48,170,50,54,6,2247,76.35,9,44,0.880000
4747,IND,M S Dhoni,10,10,71,50.72,314,183,289,341,82,10500,87.56,120,207,0.716263
3528,ENG,Joe Root,4,14,30,50.48,61,133,124,132,19,5300,87.16,0,105,0.846774


In [110]:
trace1 =go.Scatter(
                    x = consistent["Runs"],
                    y = consistent["Avg"],
                    mode = "markers",
                    name = "Name",
                    marker = dict(color=(
            ((consistent["Avg"]-(consistent["Runs"]/1000)*1.641-37.5)>0)
        ).astype('int'),
        colorscale=[[0, 'yellow'], [1, 'red']]),
                    text= consistent["player_name"])

data = [trace1]
layout = dict(title = 'The Consistent Runmakers of ODI Cricket',
              xaxis= dict(title= 'Runs Scored',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Average',ticklen= 5,zeroline= False),
              hovermode= 'closest',
             )
fig = dict(data = data, layout = layout)
iplot(fig)

<h3>Highest ODI centuries</h3>
Let's see who has got the most 100's in history

In [111]:
h100 = op.sort_values(by='100s',ascending=False).head(30)
h100.head(10)

,country,player_name,0s,100s,50s,Avg,Ca,HS,Inns,Mat,NO,Runs,S/R,St,Outs,O/I
4212,IND,Sachin Tendulkar,20,49,96,44.83,140,200,452,463,41,18426,86.24,0,411,0.909292
4951,IND,Virat Kohli,12,41,49,59.58,111,183,219,227,37,10843,92.96,0,182,0.831050
268,AUS,Ricky Ponting,20,30,82,42.04,160,164,365,375,39,13704,80.39,0,326,0.893151
10184,SRL,Sanath Jayasuriya,34,28,68,32.36,123,189,433,445,18,13430,91.20,0,415,0.958430
9345,SAF,Hashim Amla,4,27,37,49.75,83,159,171,174,12,7910,89.23,0,159,0.929825
10621,SRL,Kumar Sangakkara,15,25,93,41.99,402,169,380,404,41,14234,78.86,99,339,0.892105
12370,WIN,Chris Gayle,24,25,51,38.16,120,215,283,289,17,10151,87.14,0,266,0.939929
9702,SAF,A B de Villiers,7,25,53,53.50,176,176,218,228,39,9577,101.10,5,179,0.821101
4519,IND,Rohit Sharma,13,22,41,47.40,73,264,200,206,31,8010,87.95,0,169,0.845000
4906,IND,Sourav Ganguly,16,22,72,41.02,100,183,300,311,23,11363,73.71,0,277,0.923333


<h3>The Toppers in ODI half-centuries</h3>
Let's see who has got the most 50's in history

In [113]:
h50 = op.sort_values(by='50s',ascending=False).head(30)
h50.head(10)

,country,player_name,0s,100s,50s,Avg,Ca,HS,Inns,Mat,NO,Runs,S/R,St,Outs,O/I
4212,IND,Sachin Tendulkar,20,49,96,44.83,140,200,452,463,41,18426,86.24,0,411,0.909292
10621,SRL,Kumar Sangakkara,15,25,93,41.99,402,169,380,404,41,14234,78.86,99,339,0.892105
9146,SAF,Jacques Kallis,17,17,86,44.36,131,139,314,328,53,11579,72.89,0,261,0.831210
7924,PAK,Inzamam-ul-Haq,20,10,83,39.53,113,137,350,378,53,11739,74.24,0,297,0.848571
4156,IND,Rahul Dravid,13,12,83,39.17,196,153,318,344,40,10889,71.25,14,278,0.874214
268,AUS,Ricky Ponting,20,30,82,42.04,160,164,365,375,39,13704,80.39,0,326,0.893151
9903,SRL,Mahela Jayawardene,28,19,77,33.38,218,144,418,448,39,12650,78.96,0,379,0.906699
4906,IND,Sourav Ganguly,16,22,72,41.02,100,183,300,311,23,11363,73.71,0,277,0.923333
4747,IND,M S Dhoni,10,10,71,50.72,314,183,289,341,82,10500,87.56,120,207,0.716263
10184,SRL,Sanath Jayasuriya,34,28,68,32.36,123,189,433,445,18,13430,91.20,0,415,0.958430


In [104]:
trace2 = go.Bar(x = h100["player_name"],
                y = h100["100s"],
                name = "ODI Centuries",
                marker = dict(color = 'red',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = h100.country)
trace1 = go.Bar(
                x = h100["player_name"],
                y = h100["50s"],
                name = "ODI 50s",
                marker = dict(color = 'blue',
                              line=dict(color='rgb(0,0,0)',width=1.5)),
                text = h50.country)
data = [trace1, trace2]
layout = go.Layout(barmode = "group",hovermode= 'closest', title = "Top 30 ODI Century scorers", yaxis= dict(title= 'Num 50 : Num 100',ticklen= 5,zeroline= False))
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [105]:
trace2 = go.Bar(x = h50["player_name"],
                y = h50["100s"],
                name = "ODI Centuries",
                marker = dict(color = 'red',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = h100.country)
trace1 = go.Bar(
                x = h50["player_name"],
                y = h50["50s"],
                name = "ODI 50s",
                marker = dict(color = 'blue',
                              line=dict(color='rgb(0,0,0)',width=1.5)),
                text = h50.country)
data = [trace1, trace2]
layout = go.Layout(barmode = "group",hovermode= 'closest', title = "Top 30 ODI Half-Century scorers", yaxis= dict(title= 'Num 50 : Num 100',ticklen= 5,zeroline= False))
fig = go.Figure(data = data, layout = layout)
iplot(fig)

WORK CURRENTLY IN PROGRESS

<h3>Quack! Quack!</h3>
Let's see who has got the most ducks in history

In [164]:
ducks = op.loc[(op["0s"]>10)].sort_values(by='0s',ascending=False)
ducks.head(10)

,country,player_name,0s,100s,50s,Avg,Ca,HS,Inns,Mat,NO,Runs,S/R,St,Outs,O/I
10184,SRL,Sanath Jayasuriya,34,28,68,32.36,123,189,433,445,18,13430,91.20,0,415,0.958430
7480,PAK,Shahid Afridi,30,6,39,23.58,127,124,369,398,27,8064,117.01,0,342,0.926829
9903,SRL,Mahela Jayawardene,28,19,77,33.38,218,144,418,448,39,12650,78.96,0,379,0.906699
8429,PAK,Wasim Akram,28,0,6,16.52,88,86,280,356,55,3717,88.29,0,225,0.803571
10218,SRL,Lasith Malinga,26,0,1,7.05,31,56,113,218,35,550,75.65,0,78,0.690265
10752,SRL,Chaminda Vaas,25,0,1,13.68,60,50,220,322,72,2025,72.35,0,148,0.672727
10249,SRL,Muttiah Muralitharan,25,0,0,6.81,130,33,162,350,63,674,77.56,0,99,0.611111
12370,WIN,Chris Gayle,24,25,51,38.16,120,215,283,289,17,10151,87.14,0,266,0.939929
10389,SRL,Romesh Kaluwitharana,24,2,23,22.22,132,102,181,189,14,3711,77.70,75,167,0.922652
6631,NZL,Daniel Vettori,23,0,4,17.33,88,83,187,295,57,2253,82.98,0,130,0.695187


In [165]:
trace1 =go.Scatter(
                    x = ducks["Inns"],
                    y = ducks["0s"],
                    mode = "markers",
                    name = "Name",
                    marker = dict(color=(
            (((ducks["0s"]/ducks["Inns"])-10/100)>0)
        ).astype('int'),
        colorscale=[[0, 'red'], [1, 'yellow']]),
                    text= ducks["player_name"])

data = [trace1]
layout = dict(title = 'Getting Out at Nought',
              xaxis= dict(title= 'Number of Innings Played',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Ducks',ticklen= 5,zeroline= False),
              hovermode= 'closest',
             )
fig = dict(data = data, layout = layout)
iplot(fig)

<h3> Stay and Play Batsmen </h3>
Let's figure out which batsmen have the mentality to stay and finish the game

In [166]:
stay_play = op.loc[op["Inns"]>100].sort_values(by='O/I',ascending=True)
stay_play.head(10)

,country,player_name,0s,100s,50s,Avg,Ca,HS,Inns,Mat,NO,Runs,S/R,St,Outs,O/I
444,AUS,Brett Lee,16,0,3,17.82,54,59,110,221,44,1176,83.58,0,66,0.600000
10249,SRL,Muttiah Muralitharan,25,0,0,6.81,130,33,162,350,63,674,77.56,0,99,0.611111
9847,SAF,Lance Klusener,13,2,19,41.10,35,103,137,171,50,3576,89.92,0,87,0.635036
12674,ZIM,Prosper Utseya,16,0,4,16.74,50,68,132,164,48,1406,58.32,0,84,0.636364
9176,SAF,Shaun Pollock,20,1,14,26.46,108,130,205,303,72,3519,86.70,0,133,0.648780
4059,IND,Zaheer Khan,15,0,0,12.00,43,34,101,200,35,792,73.47,0,66,0.653465
12893,ZIM,Heath Streak,6,0,13,28.30,46,79,159,189,55,2943,73.43,0,104,0.654088
4036,IND,Anil Kumble,18,0,0,10.54,85,26,136,271,47,938,61.07,0,89,0.654412
828,AUS,Michael Bevan,5,6,46,53.58,69,108,196,232,67,6912,74.16,0,129,0.658163
6965,NZL,Kyle Mills,8,0,2,15.63,42,54,101,170,34,1047,81.67,0,67,0.663366


In [176]:
trace1 =go.Scatter(
                    x = stay_play["Runs"],
                    y = stay_play["O/I"],
                    mode = "markers",
                    name = "Name",
                    marker = dict(color=(
            ((stay_play["O/I"]<0.75)&(stay_play["Runs"]>5000))
        ).astype('int'),
        colorscale=[[0, 'yellow'], [1, 'red']]),
                    text= stay_play["player_name"])

data = [trace1]
layout = dict(title = 'Batsmen who try and stay till the end',
              xaxis= dict(title= 'Runs',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Outs/Innings',ticklen= 5,zeroline= False),
              hovermode= 'closest',
             )
fig = dict(data = data, layout = layout)
iplot(fig)

<h3>The Great Run Scorers of ODI Cricket</h3>

We are putting the number of ODI runs a batsman has scored against the number of innings he has taken to score it.

In [178]:
runs = op.loc[op["Runs"]>1000]

In [186]:
trace1 =go.Scatter(
                    x = runs["Runs"],
                    y = runs["Inns"],
                    mode = "markers",
                    name = "Name",
                    marker = dict(color=(
            (((runs["Inns"]*1000)/(runs["Runs"]))<27)
        ).astype('int'),
        colorscale=[[0, 'yellow'], [1, 'red']]),
                    text= runs["player_name"])

data = [trace1]
layout = dict(title = 'The Great Run Scorers of ODI Cricket',
              xaxis= dict(title= 'Runs',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Innings',ticklen= 5,zeroline= False),
              hovermode= 'closest',
             )
fig = dict(data = data, layout = layout)
iplot(fig)